In [1]:

#coding=utf-8
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import argparse
import numpy as np 
from keras import *
from keras.models import Sequential  
from keras.layers import *
from keras.utils.np_utils import to_categorical  
from keras.preprocessing.image import img_to_array  
from keras.callbacks import ModelCheckpoint  
from Models.utils import *

from sklearn.preprocessing import LabelEncoder  
from PIL import Image  
import matplotlib.pyplot as plt  
import cv2
import random
import os
from tqdm import tqdm  
from keras import backend as K 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import time
import gdal
seed = 7  
np.random.seed(seed)  

Using TensorFlow backend.


In [2]:
# data for training  
def generateData(batch_size,img_w,img_h,n_label,image_names=[],label_names=[]): 
    print ('gen-Sub-Image-Data...')
    image_filepath ='D:\Python\seg-data\data_MB/'
    batch_num=0
    while True:   
        bs=batch_size
        
        dataset = gdal.Open(image_filepath+image_names[batch_num%len(image_names)])
        im_width = dataset.RasterXSize #栅格矩阵的列数
        im_height = dataset.RasterYSize #栅格矩阵的行数
        image_data = dataset.ReadAsArray(0,0,im_width,im_height)
        label_data=cv2.imread(image_filepath+label_names[batch_num%len(image_names)],cv2.IMREAD_GRAYSCALE)
        train_data = []  
        train_label =  []  
        for i in range(bs):
            random_width = random.randint(0, im_width - img_w - 1)
            random_height = random.randint(0, im_height - img_h - 1)
            bands_roi=[]
            for band in image_data :
                band_roi = band[random_height: random_height + img_h, random_width: random_width + img_w]
                bands_roi.append(band_roi)
            data_roi=bands_roi
            #to_categorical(train_label, num_classes=n_label)  
            label_roi = to_categorical((label_data[random_height: random_height + img_h, random_width: random_width + img_w]).flatten(), num_classes=n_label)
            train_data.append( data_roi)  
            train_label.append(label_roi)
        #yield(np.array(train_data).shape,np.array(train_label).shape)    
        yield(np.array(train_data),np.array(train_label))
        batch_num=batch_num+1
#image_names_set=['test.tif']
#label_names_set=['test_label.png']
#for i in(generateData(8,256,256,2,image_names_set,label_names_set)):
#    print(i)
 



In [3]:
def generateDataTF(batch_size,img_w,img_h,n_label,image_names=[],label_names=[]): 
    print ('gen-Sub-Image-Data...')
    image_filepath ='D:\Python\seg-data\data_MB/'
    batch_num=0
    while True:   
        bs=batch_size
        
        dataset = gdal.Open(image_filepath+image_names[batch_num%len(image_names)])
        im_width = dataset.RasterXSize #栅格矩阵的列数
        im_height = dataset.RasterYSize #栅格矩阵的行数
        image_data = dataset.ReadAsArray(0,0,im_width,im_height)
        label_data=cv2.imread(image_filepath+label_names[batch_num%len(image_names)],cv2.IMREAD_GRAYSCALE)
        train_data = []  
        train_label =  []  
        for i in range(bs):
            random_width = random.randint(0, im_width - img_w - 1)
            random_height = random.randint(0, im_height - img_h - 1)
            bands_roi=[]
            for band in image_data :
                band_roi = band[random_height: random_height + img_h, random_width: random_width + img_w]
                bands_roi.append(band_roi)
                
            data_roi=cv2.merge(bands_roi)
            #to_categorical(train_label, num_classes=n_label)  
            label_roi = to_categorical((label_data[random_height: random_height + img_h, random_width: random_width + img_w]).flatten(), num_classes=n_label)
            train_data.append( data_roi)  
            train_label.append(label_roi)
        #yield(np.array(train_data).shape,np.array(train_label).shape)    
        yield(np.array(train_data),np.array(train_label))
        batch_num=batch_num+1
#image_names_set=['test.tif']
#label_names_set=['test_label.png']
#for i in(generateDataTF(8,256,256,2,image_names_set,label_names_set)):
#    print(i)

In [6]:
def SegNet(
        input_shape=(256,256,4),
        n_labels=2,
        kernel=3,
        pool_size=(2, 2),
        output_mode="softmax"):
    # encoder
    inputs = Input(shape=input_shape)
    img_w=input_shape[1]
    img_h=input_shape[2] 
    conv_1 = Convolution2D(64, (kernel, kernel), padding="same")(inputs)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation("relu")(conv_1)
    conv_2 = Convolution2D(64, (kernel, kernel), padding="same")(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = Activation("relu")(conv_2)

    pool_1, mask_1 = MaxPoolingWithArgmax2D(pool_size)(conv_2)

    conv_3 = Convolution2D(128, (kernel, kernel), padding="same")(pool_1)
    conv_3 = BatchNormalization()(conv_3)
    conv_3 = Activation("relu")(conv_3)
    conv_4 = Convolution2D(128, (kernel, kernel), padding="same")(conv_3)
    conv_4 = BatchNormalization()(conv_4)
    conv_4 = Activation("relu")(conv_4)

    pool_2, mask_2 = MaxPoolingWithArgmax2D(pool_size)(conv_4)

    conv_5 = Convolution2D(256, (kernel, kernel), padding="same")(pool_2)
    conv_5 = BatchNormalization()(conv_5)
    conv_5 = Activation("relu")(conv_5)
    conv_6 = Convolution2D(256, (kernel, kernel), padding="same")(conv_5)
    conv_6 = BatchNormalization()(conv_6)
    conv_6 = Activation("relu")(conv_6)
    conv_7 = Convolution2D(256, (kernel, kernel), padding="same")(conv_6)
    conv_7 = BatchNormalization()(conv_7)
    conv_7 = Activation("relu")(conv_7)

    pool_3, mask_3 = MaxPoolingWithArgmax2D(pool_size)(conv_7)

    conv_8 = Convolution2D(512, (kernel, kernel), padding="same")(pool_3)
    conv_8 = BatchNormalization()(conv_8)
    conv_8 = Activation("relu")(conv_8)
    conv_9 = Convolution2D(512, (kernel, kernel), padding="same")(conv_8)
    conv_9 = BatchNormalization()(conv_9)
    conv_9 = Activation("relu")(conv_9)
    conv_10 = Convolution2D(512, (kernel, kernel), padding="same")(conv_9)
    conv_10 = BatchNormalization()(conv_10)
    conv_10 = Activation("relu")(conv_10)

    pool_4, mask_4 = MaxPoolingWithArgmax2D(pool_size)(conv_10)

    conv_11 = Convolution2D(512, (kernel, kernel), padding="same")(pool_4)
    conv_11 = BatchNormalization()(conv_11)
    conv_11 = Activation("relu")(conv_11)
    conv_12 = Convolution2D(512, (kernel, kernel), padding="same")(conv_11)
    conv_12 = BatchNormalization()(conv_12)
    conv_12 = Activation("relu")(conv_12)
    conv_13 = Convolution2D(512, (kernel, kernel), padding="same")(conv_12)
    conv_13 = BatchNormalization()(conv_13)
    conv_13 = Activation("relu")(conv_13)

    pool_5, mask_5 = MaxPoolingWithArgmax2D(pool_size)(conv_13)
    print("Build enceder done..")

    # decoder

    unpool_1 = MaxUnpooling2D(pool_size)([pool_5, mask_5])

    conv_14 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_1)
    conv_14 = BatchNormalization()(conv_14)
    conv_14 = Activation("relu")(conv_14)
    conv_15 = Convolution2D(512, (kernel, kernel), padding="same")(conv_14)
    conv_15 = BatchNormalization()(conv_15)
    conv_15 = Activation("relu")(conv_15)
    conv_16 = Convolution2D(512, (kernel, kernel), padding="same")(conv_15)
    conv_16 = BatchNormalization()(conv_16)
    conv_16 = Activation("relu")(conv_16)

    unpool_2 = MaxUnpooling2D(pool_size)([conv_16, mask_4])

    conv_17 = Convolution2D(512, (kernel, kernel), padding="same")(unpool_2)
    conv_17 = BatchNormalization()(conv_17)
    conv_17 = Activation("relu")(conv_17)
    conv_18 = Convolution2D(512, (kernel, kernel), padding="same")(conv_17)
    conv_18 = BatchNormalization()(conv_18)
    conv_18 = Activation("relu")(conv_18)
    conv_19 = Convolution2D(256, (kernel, kernel), padding="same")(conv_18)
    conv_19 = BatchNormalization()(conv_19)
    conv_19 = Activation("relu")(conv_19)

    unpool_3 = MaxUnpooling2D(pool_size)([conv_19, mask_3])

    conv_20 = Convolution2D(256, (kernel, kernel), padding="same")(unpool_3)
    conv_20 = BatchNormalization()(conv_20)
    conv_20 = Activation("relu")(conv_20)
    conv_21 = Convolution2D(256, (kernel, kernel), padding="same")(conv_20)
    conv_21 = BatchNormalization()(conv_21)
    conv_21 = Activation("relu")(conv_21)
    conv_22 = Convolution2D(128, (kernel, kernel), padding="same")(conv_21)
    conv_22 = BatchNormalization()(conv_22)
    conv_22 = Activation("relu")(conv_22)

    unpool_4 = MaxUnpooling2D(pool_size)([conv_22, mask_2])

    conv_23 = Convolution2D(128, (kernel, kernel), padding="same")(unpool_4)
    conv_23 = BatchNormalization()(conv_23)
    conv_23 = Activation("relu")(conv_23)
    conv_24 = Convolution2D(64, (kernel, kernel), padding="same")(conv_23)
    conv_24 = BatchNormalization()(conv_24)
    conv_24 = Activation("relu")(conv_24)

    unpool_5 = MaxUnpooling2D(pool_size)([conv_24, mask_1])

    conv_25 = Convolution2D(64, (kernel, kernel), padding="same")(unpool_5)
    conv_25 = BatchNormalization()(conv_25)
    conv_25 = Activation("relu")(conv_25)

    conv_26 = Convolution2D(n_labels, (1, 1), padding="same")(conv_25)
    conv_26 = BatchNormalization()(conv_26)
   
    conv_26  = Reshape((n_labels,-1))(conv_26 )

    conv_26  = Permute((2,1))(conv_26 )
    outputs = Activation(output_mode)(conv_26)

    model = Model(inputs=inputs, outputs=outputs, name="SegNet")
    model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

    return model

In [7]:
def train(): 
    EPOCHS = 1
    BS = 2
    img_w = 256  
    img_h = 256  
    model = SegNet() 
    model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])  
    
    modelcheck = ModelCheckpoint('..\..\Python\seg-data/model/4Bands-'+time.strftime(f'%Y-%m-%d-%a-%H-%M-%S',time.localtime(time.time()))+'.h5',monitor='val_acc',save_best_only=True,mode='max')  
    
    callable = [modelcheck]  
    
    train_numb = 1000
    valid_numb = 200
    print ("the number of train data is",train_numb,train_numb//BS)  
    print ("the number of val data is",valid_numb,valid_numb//BS)
    H = model.fit_generator(generator=generateDataTF(BS,img_w,img_h,2,['test.tif'],['test_label.png']),
                            steps_per_epoch=train_numb//BS,
                            epochs=EPOCHS,
                            verbose=1,
                            validation_data=generateDataTF(BS,img_w,img_h,2,['test.tif'],['test_label.png']),
                            validation_steps=valid_numb//BS,
                            callbacks=callable,
                            max_q_size=1)  

    # plot the training loss and accuracy
    plt.style.use("ggplot")
    plt.figure()
    N = EPOCHS
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy on SegNet Satellite Seg")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig("plot.png")



def args_parse():
    # construct the argument parse and parse the arguments
    ap = argparse.ArgumentParser()
    ap.add_argument("-a", "--augment",help="using data augment or not",
                    action="store_true", default=False)
    ap.add_argument("-m", "--model", required=True,default='model/'+time.strftime(f'%Y-%m-%d %a %H:%M:%S',time.localtime(time.time()))+'.h5'
                    ,help="path to output model")
    ap.add_argument("-p", "--plot", type=str, default="plot.png",
                    help="path to output accuracy/loss plot")
    args = vars(ap.parse_args()) 
    return args


train()

Build enceder done..
Tensor("max_unpooling2d_6/max_unpooling2d_6/Size_1:0", shape=(), dtype=int32) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Tensor("max_unpooling2d_7/max_unpooling2d_7/Size_1:0", shape=(), dtype=int32) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Tensor("max_unpooling2d_8/max_unpooling2d_8/Size_1:0", shape=(), dtype=int32) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Tensor("max_unpooling2d_9/max_unpooling2d_9/Size_1:0", shape=(), dtype=int32) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Tensor("max_unpooling2d_10/max_unpooling2d_10/Size_1:0", shape=(), dtype=int32) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
the number of train data is 1000 500
the number of val data is 200 100


c:\users\chao\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., steps_per_epoch=500, epochs=1, verbose=1, validation_data=<generator..., validation_steps=100, callbacks=[<keras.ca..., max_queue_size=1)`


Epoch 1/1
gen-Sub-Image-Data...gen-Sub-Image-Data...



203/500 [===========>..................] - ETA: 44:14 - loss: 0.8983 - acc: 0.49 - ETA: 23:06 - loss: 0.9079 - acc: 0.48 - ETA: 16:00 - loss: 0.8943 - acc: 0.48 - ETA: 12:30 - loss: 0.8932 - acc: 0.48 - ETA: 10:20 - loss: 0.8792 - acc: 0.49 - ETA: 8:55 - loss: 0.8792 - acc: 0.4932 - ETA: 7:55 - loss: 0.8783 - acc: 0.493 - ETA: 7:10 - loss: 0.8756 - acc: 0.493 - ETA: 6:34 - loss: 0.8804 - acc: 0.492 - ETA: 6:06 - loss: 0.8779 - acc: 0.493 - ETA: 5:42 - loss: 0.8719 - acc: 0.495 - ETA: 5:22 - loss: 0.8748 - acc: 0.494 - ETA: 5:06 - loss: 0.8711 - acc: 0.495 - ETA: 4:51 - loss: 0.8735 - acc: 0.494 - ETA: 4:39 - loss: 0.8687 - acc: 0.495 - ETA: 4:27 - loss: 0.8698 - acc: 0.495 - ETA: 4:18 - loss: 0.8667 - acc: 0.495 - ETA: 4:09 - loss: 0.8670 - acc: 0.495 - ETA: 4:01 - loss: 0.8662 - acc: 0.496 - ETA: 3:54 - loss: 0.8642 - acc: 0.496 - ETA: 3:48 - loss: 0.8641 - acc: 0.496 - ETA: 3:43 - loss: 0.8610 - acc: 0.497 - ETA: 3:38 - loss: 0.8581 - acc: 0.498 - ETA: 3:33 - loss: 0.8611 - acc: 0.49

415/500 [=======================>......] - ETA: 1:17 - loss: 0.7121 - acc: 0.598 - ETA: 1:17 - loss: 0.7123 - acc: 0.598 - ETA: 1:17 - loss: 0.7114 - acc: 0.599 - ETA: 1:17 - loss: 0.7095 - acc: 0.601 - ETA: 1:16 - loss: 0.7076 - acc: 0.602 - ETA: 1:16 - loss: 0.7063 - acc: 0.604 - ETA: 1:16 - loss: 0.7053 - acc: 0.604 - ETA: 1:15 - loss: 0.7037 - acc: 0.606 - ETA: 1:15 - loss: 0.7026 - acc: 0.606 - ETA: 1:15 - loss: 0.7011 - acc: 0.608 - ETA: 1:15 - loss: 0.6996 - acc: 0.609 - ETA: 1:14 - loss: 0.7000 - acc: 0.609 - ETA: 1:14 - loss: 0.6987 - acc: 0.610 - ETA: 1:14 - loss: 0.6973 - acc: 0.611 - ETA: 1:13 - loss: 0.6962 - acc: 0.612 - ETA: 1:13 - loss: 0.6946 - acc: 0.613 - ETA: 1:13 - loss: 0.6933 - acc: 0.614 - ETA: 1:12 - loss: 0.6920 - acc: 0.616 - ETA: 1:12 - loss: 0.6907 - acc: 0.617 - ETA: 1:12 - loss: 0.6894 - acc: 0.618 - ETA: 1:12 - loss: 0.6896 - acc: 0.618 - ETA: 1:11 - loss: 0.6878 - acc: 0.620 - ETA: 1:11 - loss: 0.6870 - acc: 0.620 - ETA: 1:11 - loss: 0.6856 - acc: 0.621

450/500 [==========================>...] - ETA: 21s - loss: 0.5533 - acc: 0.72 - ETA: 20s - loss: 0.5524 - acc: 0.72 - ETA: 20s - loss: 0.5515 - acc: 0.72 - ETA: 20s - loss: 0.5518 - acc: 0.72 - ETA: 20s - loss: 0.5515 - acc: 0.72 - ETA: 19s - loss: 0.5506 - acc: 0.72 - ETA: 19s - loss: 0.5499 - acc: 0.72 - ETA: 19s - loss: 0.5490 - acc: 0.72 - ETA: 19s - loss: 0.5485 - acc: 0.72 - ETA: 18s - loss: 0.5477 - acc: 0.72 - ETA: 18s - loss: 0.5468 - acc: 0.73 - ETA: 18s - loss: 0.5462 - acc: 0.73 - ETA: 18s - loss: 0.5460 - acc: 0.73 - ETA: 17s - loss: 0.5454 - acc: 0.73 - ETA: 17s - loss: 0.5445 - acc: 0.73 - ETA: 17s - loss: 0.5440 - acc: 0.73 - ETA: 17s - loss: 0.5447 - acc: 0.73 - ETA: 16s - loss: 0.5446 - acc: 0.73 - ETA: 16s - loss: 0.5439 - acc: 0.73 - ETA: 16s - loss: 0.5434 - acc: 0.73 - ETA: 16s - loss: 0.5425 - acc: 0.73 - ETA: 15s - loss: 0.5418 - acc: 0.73 - ETA: 15s - loss: 0.5414 - acc: 0.73 - ETA: 15s - loss: 0.5409 - acc: 0.73 - ETA: 15s - loss: 0.5400 - acc: 0.73 - ETA: 14

KeyboardInterrupt: 

In [ ]:
m = SegNet()
from keras.utils import plot_model
plot_model(m, show_shapes=True, to_file='model_segnet.png')

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline 